# The RFSoC QSFP offload client notebook
---
<div class="alert alert-box alert-info">
Please use Jupyter labs http://board_ip_address/lab for this notebook.
</div>

## Overview
> The RFSoC QSFP offload client notebook contains an example implementation of an UDP client to receive data from the RFSoC4x2 board. We can receive data from either the RF-ADC or the PS via the QSFP connection. In this notebook we first receive data from the RF-ADC and plot it in the frequency domain. We then go on to receive data from the PS.

## Aims
* To receive data from the RFSoC through QSFP network interface.
* To act as an example of UDP client implementation.
* Demonstrate compatibility with Python UDP socket interface.

## Last Revised
* 08/07/22 - Updated code and text
* 30 June 2022 - Initial Revision
---

This overlay requires two notebooks to operate: a board notebook, to be run on the RFSoC board, and a client notebook (this one), which must be run on the PC/server that is connected to the board via the QSFP28 connection.

Before we start, make sure this notebook is downloaded onto your PC/server and opened in Jupyter-Lab. 

We will be jumping between the board and client notebooks throughout this example, so it is best to have them both open in separate browser windows. In this notebook any commands that are required to be run on the board side will be preceded by a <span style="color:orange">**orange**</span> alert box, while any commands to be run on the client will be preceded by a <span style="color:green">**green**</span> alert box. All main section headings are identical between board and client notebooks and are numbered to make it easier to follow through the different steps.

---

<div class="alert alert-box alert-warning">
The following steps are to be executed on the board.
</div>

## 1. Board Setup

Run the cells in the board notebook to download the overlay and setup the CMAC.

---

<div class="alert alert-box alert-success">
The following steps are to be executed on the client.
</div>

## 2. Client Setup

Set up the QSFP NIC on the PC/server to have a static IP address. In this example we use `192.168.4.1` for the client side, and `192.168.4.99` for the board. Change accordingly if these conflict with any other IPs on your network.

Run the supplied `qsfp_setup.sh` script on the client side (with super user privilages) or manually configure your QSFP network card for:

* Link Mode to forced 100G
* FEC to RS (RS-FEC)
* FEC speed to 100G

Additionally, if available, set Maximum Transmission Unit (MTU) to 9000 in your network manager settings. This will allow the use of jumbo frames.

---

<div class="alert alert-box alert-warning">
The following steps are to be executed on the board.
</div>

## 3. Configuring the Overlay

On the board, set up the Netlayer IP to configure the network settings and set up a socket, then set up the RF data converters. 

---

<div class="alert alert-box alert-success">
The following steps are to be executed on the client.
</div>

## 4. Receiving RF Data

We can now set up the connection to the socket and start receving data!

### Setup the Socket

First we need to connect to the open UDP socket on the RFSoC4x2 development board. 

In [ ]:
import socket

udp_port = 60133
client_ip = "192.168.4.1"

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.settimeout(3)
s.bind((client_ip, udp_port))

### Getting the Data

To receive data we need to first grab a UDP packet, then perform an FFT so we can visualise the data in the frequency domain. To do this we create three helper functions, `get_data()`, `de_interleave()`, and `compute_fft()`.

The `get_data()` function first grabs a UDP frame and converts the buffer from bytes to 16-bit integers. 

The `de_interleave()` function de-interleaves the data, separating out the I and Q (real and imaginary) components.

The `compute_fft()` function applies a window to the I and Q signals and then performs an FFT.

In [ ]:
import numpy as np

def compute_fft(data):
    # Apply a window to the data first
    re = data[0]*np.blackman(data[0].size)
    im = data[1]*np.blackman(data[1].size)
    
    # Compute FFT if data is not zero-valued
    if (np.any(re) or np.any(im)):
        fdata = np.fft.fftshift(np.fft.fft(re+1j*im))
        pdata = abs(fdata)
        ldata = 20*np.where(pdata>0, np.log10(pdata), np.log10(2**-15))
        normalised = ldata - ldata.max() # dBc
        return normalised
    # Otherwise just return zero-valued data
    else:
        return re
    
def de_interleave(data):
    re_int16 = data[::2]
    im_int16 = data[1::2]
    
    return (re_int16, im_int16)
    
def get_data(num_bytes):
    # Attempt to grab data.
    try:
        data = s.recv(num_bytes)
    # If socket timeout then just return zero values
    except socket.error:
        data = np.zeros(num_bytes, dtype=np.uint8)
    
    # Format data buffer into int16s
    iq_int16 = np.frombuffer(data, dtype=np.int16)
    
    return iq_int16

Now we can create a figure object using Plotly and fill it with an initial frame of data

In [ ]:
import plotly.graph_objs as go

fs = 2457.6e6

y_data = compute_fft(de_interleave(get_data(9000)))
x_data = np.linspace(-fs/2, fs/2, y_data.size, endpoint=False) + 1228.8e6

fft_fig = go.FigureWidget(go.Scatter(y=y_data, x=x_data))

Next we need to set up a method of streaming the data continuously. For this we use the Python `threading` library to set up a concurrent process. This allows us to still use this notebook while we are grabbing and visualising the data. Below we create four functions: `update_fig()`, `do()`, `start()`, and `stop()`.

`update_fig()` determines how often we grab data and update the plot. The `t` variable at the top of the cell is set to 0.2, which means a new UDP packet is retreived every 200 ms. The `num_bytes` variable at the top of the cell determines how maby bytes of data we receive for each packet. For jumbo frames use a buffer size of 9000, otherwise use 1024.

The `start()` and `stop()` functions allow us to quickly and easily start and stop the processes that run in `do()`. `start()` is also responsible for creating a concurrent thread.

In [ ]:
t = 0.2 # seconds
num_bytes = 9000 # bytes

import threading
import time

stopping = True # do not change this variable

def update_fig():
    while not stopping:
        next_timer = time.time() + t
        data = de_interleave(get_data(num_bytes))
        data_fft = compute_fft(data)
        fft_fig.data[0]['y'] = data_fft
        sleep_time = next_timer - time.time()
        if sleep_time > 0:
            time.sleep(sleep_time)
            
def start():
    global stopping
    if stopping:
        stopping = False
        thread = threading.Thread(target=update_fig)
        thread.start()
        
def stop():
    global stopping
    stopping = True

### Plot the received data

Now we've set up our methods to receive, format, process and stream the data, we can now plot it in the frequency domain.

First we draw the figure we created earlier.

In [ ]:
x_ticks = [400e6, 800e6, 1200e6, 1600e6, 2000e6, fs]
fft_fig.update_layout(
    yaxis_title="Magnitude (dBc)",
    xaxis_title="Frequency (MHz)",
    yaxis_range=[-90,5],
    xaxis_range=[0,fs],
    xaxis_tickvals=x_ticks,
    xaxis_ticktext=["{:.1f}".format(i/1e6) for i in x_ticks]
)

Now we can use the `start()` function to continuously stream the data!

In [ ]:
start()

---

<div class="alert alert-box alert-warning">
The following steps are to be executed on the board.
</div>

## 5. Generating a Signal

Return to the board notebook and generate a signal to send to the RF-ADC. The plot should automatically update when the signal is received. Note that it may take a few seconds before a signal shows as the old data is cleared from the buffer. Remember, there's a lot of data being transferred and we're not updating at full speed so not to overload the client!

---

<div class="alert alert-box alert-success">
The following steps are to be executed on the client.
</div>

## 6. Stop Receiving Data

We can now stop plotting the data from the RF-ADC and instead receive a UDP packet sent from the PS instead.

In [ ]:
stop()

---

<div class="alert alert-box alert-warning">
The following steps are to be executed on the board.
</div>

## 7. Sending Data from the PS

Follow the steps in the board notebook to generate and send the ramp signal.

---

<div class="alert alert-box alert-success">
The following steps are to be executed on the client.
</div>

## 8. Receving Data from the PS

We can now grab the data and plot it in a new figure object.

In [ ]:
y_data = get_data(num_bytes)
x_data = np.arange(y_data.size)

time_fig = go.FigureWidget(go.Scatter(y=y_data, x=x_data))
time_fig

There may be some residual data from the previous section still in the buffer so you may need to refresh the plot a few times to get the correct data. Run the cell below as many times as you need and it should update the plot automatically. 

Note that, if you run the cell too many times, there may be no data to receive, causing the socket to timeout after a few seconds. In this case the data will be written as zero values instead.

In [ ]:
y_data = get_data(num_bytes)
time_fig.data[0]['y'] = y_data

---

## GNU Radio demo
Jump to [rfsoc_qsfp/gnuradio](https://github.com/strath-sdr/rfsoc_qsfp_offload/tree/master/gnuradio) to explore the alternative GNU Radio demo.

## Conclusion

This notebook has shown how to receive data send by the RFSoC offload overlay using standard Python networking libraries.